In [ ]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
from itertools import chain
from tqdm.notebook import tqdm
from pdf2image import convert_from_path

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(torch.__version__)

In [ ]:
labels = [
    'O',
    'B-QUESTION',
    'B-ANSWER',
    'B-HEADER',
    'I-ANSWER',
    'I-QUESTION',
    'I-HEADER'
]
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}

Replace `"tokenizer_class": "XLMRobertaTokenizer"` in config.json with `"tokenizer_class": "LayoutXLMTokenizer"`

In [ ]:
from transformers import LayoutXLMProcessor, LayoutLMv2ForTokenClassification

processor = LayoutXLMProcessor.from_pretrained('./test-ner')
model = LayoutLMv2ForTokenClassification.from_pretrained("./test-ner")

In [ ]:
def unnormalize_box(bbox, width, height):
    return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
    ]

def iob_to_label(label):
    label = label[2:]
    if not label:
        return 'other'
    return label

label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}



In [ ]:
def infer(image, processor, model, label2color):
    image = image.convert("RGB")
    encoding = processor(image, return_offsets_mapping=True, return_tensors="pt", truncation=True, max_length=514)
    offset_mapping = encoding.pop('offset_mapping')
    outputs = model(**encoding)
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    token_boxes = encoding.bbox.squeeze().tolist()

    width, height = image.size
    is_subword = np.array(offset_mapping.squeeze().tolist())[:,0] != 0

    true_predictions = [id2label[pred] for idx, pred in enumerate(predictions) if not is_subword[idx]]
    true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(token_boxes) if not is_subword[idx]]
    draw = ImageDraw.Draw(image)

    font = ImageFont.load_default()

    for prediction, box in zip(true_predictions, true_boxes):
        predicted_label = iob_to_label(prediction).lower()
        draw.rectangle(box, outline=label2color[predicted_label])
        draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)

    return image

In [ ]:
imgs = [convert_from_path(img_path) for img_path in Path('./infer/pdf/').glob('*.pdf')]
imgs = list(chain.from_iterable(imgs))
len(imgs)

In [ ]:
outputs = [infer(image, processor, model, label2color) for image in tqdm(imgs)]

In [ ]:
for o in outputs:
    display(o)